In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D , Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/airline_sentiment_analysis.csv')

In [ ]:
class Model:
    def __init__(self, datafile = '/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/airline_sentiment_analysis.csv'):
        self.data = pd.read_csv(datafile)
        self.max_fatures = 2000
        self.batch_size = 128
    def split(self, test_size):
        tokenizer = Tokenizer(num_words=self.max_fatures, split=' ')
        tokenizer.fit_on_texts(self.data['text'].values)
        X = tokenizer.texts_to_sequences(self.data['text'].values)
        X = pad_sequences(X)

        y = pd.get_dummies(self.data['airline_sentiment']).values
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    def preprocessor(self):
        # Remove HTML markup
        self.data['text'] = data['text'].apply(lambda x: x.lower())
        # removing special chars
        self.data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
        self.data['text'] = data['text'].str.replace('rt','')
    def create_model(self,embed_dim=128,lstm_out = 196):
        model = Sequential()
        model.add(Embedding(self.max_fatures, embed_dim,input_length = 32))
        model.add(SpatialDropout1D(0.4))
        model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
        model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
        model.add(Dense(2,activation='softmax'))
        model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
        print(model.summary())
        return model
    def fit(self, model):
        model.fit(self.X_train, self.Y_train, epochs = 15, batch_size=self.batch_size, verbose = 1)
        return model
    def evaluate(self):
        score,acc = model.evaluate(self.X_test, self.Y_test, verbose = 2, batch_size = self.batch_size)
        print(score,acc)
    def Save_tokenizer(self):
      # saving
        with open('/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/tokenizer.pickle', 'wb') as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    def classification_report(self):
        predict_x=model.predict(self.X_test) 
        classes_x=np.argmax(predict_x,axis=1)
        df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':classes_x})
        df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
        print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
        print(classification_report(df_test.true, df_test.pred))
    def predict(self, model1,text):
        twt = [text]
        #vectorizing the tweet by the pre-fitted tokenizer instance
        twt = tokenizer1.texts_to_sequences(twt)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
        print(twt)
        sentiment = model1.predict(twt,batch_size=1,verbose = 2)[0]
        if(np.argmax(sentiment) == 0):
            print("negative")
        elif (np.argmax(sentiment) == 1):
            print("positive")

In [ ]:
if __name__ == "__main__":
  modle= Model()
  modle.preprocessor()
  modle.split(0.20)
  modle.Save_tokenizer()
  model= modle.create_model()
  modle.fit(model)
  modle.evaluate()

In [ ]:
data = data[['text','airline_sentiment']]

In [ ]:
data.head()

,text,airline_sentiment
0,@VirginAmerica plus you've added commercials t...,positive
1,@VirginAmerica it's really aggressive to blast...,negative
2,@VirginAmerica and it's a really big bad thing...,negative
3,@VirginAmerica seriously would pay $30 a fligh...,negative
4,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['text'] = data['text'].str.replace('rt','')
data.head()

,text,airline_sentiment
0,virginamerica plus youve added commercials to ...,positive
1,virginamerica its really aggressive to blast o...,negative
2,virginamerica and its a really big bad thing a...,negative
3,virginamerica seriously would pay 30 a flight ...,negative
4,virginamerica yes nearly every time i fly vx t...,positive


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9232, 32) (9232, 2)
(2309, 32) (2309, 2)


In [ ]:

# saving
with open('/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
X.shape[1]

32

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 32, 128)          0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 32, 392)          509600    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 392)              923552    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 2)                 786       
                                                        

In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 1)


Epoch 1/25
73/73 [==============================] - 167s 2s/step - loss: 0.3941 - accuracy: 0.8317
Epoch 2/25
73/73 [==============================] - 147s 2s/step - loss: 0.1916 - accuracy: 0.9254
Epoch 3/25
73/73 [==============================] - 152s 2s/step - loss: 0.1527 - accuracy: 0.9418
Epoch 4/25
73/73 [==============================] - 152s 2s/step - loss: 0.1261 - accuracy: 0.9543
Epoch 5/25
73/73 [==============================] - 149s 2s/step - loss: 0.1051 - accuracy: 0.9607
Epoch 6/25
73/73 [==============================] - 153s 2s/step - loss: 0.0923 - accuracy: 0.9657
Epoch 7/25
73/73 [==============================] - 148s 2s/step - loss: 0.0776 - accuracy: 0.9734
Epoch 8/25
73/73 [==============================] - 150s 2s/step - loss: 0.0674 - accuracy: 0.9748
Epoch 9/25
73/73 [==============================] - 150s 2s/step - loss: 0.0555 - accuracy: 0.9818
Epoch 10/25
73/73 [==============================] - 148s 2s/step - loss: 0.0495 - accuracy: 0.9814
Epoch 11/

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("acc: %.2f" % (acc))

19/19 - 8s - loss: 0.5987 - accuracy: 0.9086 - 8s/epoch - 445ms/step
acc: 0.91


In [ ]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

73/73 [==============================] - 12s 145ms/step


In [ ]:
classes_x

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':classes_x})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))

confusion matrix [[1770   92]
 [ 119  328]]


In [ ]:
print(classification_report(df_test.true, df_test.pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1862
           1       0.78      0.73      0.76       447

    accuracy                           0.91      2309
   macro avg       0.86      0.84      0.85      2309
weighted avg       0.91      0.91      0.91      2309



In [ ]:
model.save("/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/new_model1.h5")
print("Saved model to disk")

In [ ]:
with open('/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/tokenizer.pickle', 'rb') as handle:
    tokenizer1 = pickle.load(handle)


In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('/content/drive/MyDrive/TRUE FOUNDRY INTERNSHIP/model.h5')
# summarize model.

In [ ]:
twt = ['bad airline']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer1.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
print(twt)
sentiment = model1.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")


[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0 192  93]]
1/1 - 1s - 1s/epoch - 1s/step
negative


In [ ]:
twt = ['best airline']
#vectorizing the tweet by the pre-fitted tokenizer instance
max_fatures = 2000
twt = tokenizer1.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
print(twt)
sentiment = model1.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0 201  93]]
1/1 - 2s - 2s/epoch - 2s/step
positive
